In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
import os
import random
import pickle
import tqdm
import time
os.environ["PATH"] += os.pathsep + os.path.join("D:\\Graphviz\\bin")
import string
import emoji
import emojis
import re
import random
from torch.autograd import Variable
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
data = pd.read_csv('hindi_dataset/hindi_news_dataset.csv')
data.head(2)

,Headline,Content,News Categories,Date
0,कांग्रेस नेता बलजिंदर सिंह की पंजाब में घर के ...,कांग्रेस नेता बलजिंदर सिंह की सोमवार को पंजाब ...,['national'],19-09-2023
1,केंद्रीय मंत्री बोले- महिला आरक्षण लाने का साह...,केंद्रीय मंत्री प्रह्लाद पटेल ने लोकसभा और विध...,"['politics', 'national']",19-09-2023


In [3]:
data['combined_news'] = data['Headline']+ ' : '+data['Content']
data.dropna(inplace=True)
news = 124
data.iloc[news]['Headline'],data.iloc[news]['Content'],data.iloc[news]['combined_news']

('बिहार में ₹5 लाख दहेज न देने पर गर्भवती महिला को पति व सास-ससुर ने गर्म सलाखों से दागा',
 'न्यूज़18 के मुताबिक, गोपालगंज (बिहार) में ₹5 लाख दहेज न देने पर 6-महीने की गर्भवती महिला को पति व सास-ससुर ने कथित तौर पर पीटा व उसके शरीर को गर्म सलाखों से दाग दिया। रिपोर्ट के मुताबिक, ससुरालवालों ने महिला को ज़िंदा जलाने का भी प्रयास किया। पीड़िता की शिकायत पर पुलिस ने मामले में एफआईआर दर्ज कर ली है। \n\n\n\n',
 'बिहार में ₹5 लाख दहेज न देने पर गर्भवती महिला को पति व सास-ससुर ने गर्म सलाखों से दागा : न्यूज़18 के मुताबिक, गोपालगंज (बिहार) में ₹5 लाख दहेज न देने पर 6-महीने की गर्भवती महिला को पति व सास-ससुर ने कथित तौर पर पीटा व उसके शरीर को गर्म सलाखों से दाग दिया। रिपोर्ट के मुताबिक, ससुरालवालों ने महिला को ज़िंदा जलाने का भी प्रयास किया। पीड़िता की शिकायत पर पुलिस ने मामले में एफआईआर दर्ज कर ली है। \n\n\n\n')

In [4]:
dataset = list(data['combined_news'])
import re
pattern = re.compile("["
                       '\u200b'
                     '\u200d'
                       "]+", flags=re.UNICODE)
pattern.sub('', 'hello \u200b dsafsdf \u200d sadf ewf')
# since it is a character level language generation model thus converting
# examples from tokens and sentences level to characters
dataset = ' '.join(dataset)
dataset = emoji.replace_emoji(dataset)
dataset = pattern.sub('', dataset)
dataset = dataset.replace('\n',' ')
print('Number of Characters : ', len(dataset))
print(dataset[:1500])

Number of Characters :  1753406
कांग्रेस नेता बलजिंदर सिंह की पंजाब में घर के अंदर गोली मारकर की गई हत्या : कांग्रेस नेता बलजिंदर सिंह की सोमवार को पंजाब के मोगा में उनके घर में गोली मारकर हत्या कर दी गई। ऑनलाइन सामने आए सीसीटीवी फुटेज में बलजिंदर को गोलियां मारता हुआ एक हमलावर दिख रहा है। पुलिस ने बताया, "बलजिंदर को एक गोली सीने में लगी और आशंका है कि दूसरा हमलावर घर के बाहर बाइक पर सवार था।" केंद्रीय मंत्री बोले- महिला आरक्षण लाने का साहस मोदी सरकार में था; बाद में डिलीट किया पोस्ट : केंद्रीय मंत्री प्रह्लाद पटेल ने लोकसभा और विधानसभाओं में महिला आरक्षण को लेकर 'X' पर पोस्ट किया कि यह मांग पूरी करने का नैतिक साहस प्रधानमंत्री नरेंद्र मोदी के नेतृत्व वाली सरकार में ही था। उन्होंने आगे कहा कि केंद्रीय कैबिनेट की मंज़ूरी से यह साबित हो गया है। हालांकि, कुछ देर बाद पटेल ने अपना पोस्ट डिलीट कर दिया। ओपीएस लागू करने से अस्थिर हो सकती है राज्यों की वित्तीय स्थिति: आरबीआई अधिकारी : आरबीआई के 5 अधिकारियों ने एक लेख में लिखा है कि राज्यों द्वारा पुरानी पेंशन योजना (ओपीएस) लागू करना पीछे ले जान

In [5]:
#creating character vocabulary
class CharLevelVocabulary:
    def __init__(self,pad_token="<pad>",eos_token="<eos>",unk_token="<UNK>"):
        self.special_tokens = {"UNK":unk_token,"PAD":pad_token,
                              "EOS":eos_token}
        self.unique_chars = []
        self.char2int = {}
        
        #Adding special token in Vocabulary
        for token in self.special_tokens.values():
            if token is not None:
                self.add_char(token)
                
    def add_char(self,char):
        if char not in self.char2int:
            index = len(self.unique_chars)
            self.char2int[char] = torch.tensor(index)
            self.unique_chars.append(char)
            
    def build_vocabulary(self, text):
        for sentence in text:
            for char in sentence:
                self.add_char(char)
        

    def __call__(self, text):
        self.build_vocabulary(text)

In [6]:

vocab = CharLevelVocabulary()
vocab(dataset)
print('Length of Vocabulary : ',len(vocab.unique_chars),end='\n\n')
print('Int to char: ',vocab.unique_chars,end='\n\n')
print('Char to int: ',vocab.char2int,end='\n\n')

Length of Vocabulary :  148

Int to char:  ['<UNK>', '<pad>', '<eos>', 'क', 'ा', 'ं', 'ग', '्', 'र', 'े', 'स', ' ', 'न', 'त', 'ब', 'ल', 'ज', 'ि', 'द', 'ह', 'ी', 'प', 'म', 'घ', 'अ', 'ो', 'ई', 'य', ':', 'व', 'उ', '।', 'ऑ', 'इ', 'आ', 'ए', 'ट', 'फ', 'ु', 'ख', 'ै', ',', '"', 'औ', 'श', 'ू', 'थ', '-', 'ष', 'ण', ';', 'ड', 'भ', 'ध', 'ओ', "'", 'X', 'ृ', '़', 'छ', '5', '(', ')', 'झ', '4', '.', 'ढ', '1', 'ौ', '3', 'ॉ', 'ञ', 'ठ', '%', '0', '8', '2', 'च', '7', 'ऊ', '6', '₹', 'ज़', 'ड़', '/', '?', 'ऐ', '’', 'ढ़', '!', '9', 'ः', 'I', 'h', 't', 'p', 's', 'b', 'e', 'c', 'o', 'm', 'G', 'r', 'i', 'v', 'a', 'n', 'l', 'g', 'N', 'D', 'A', '…', 'ऋ', '$', 'k', '”', '+', 'V', '€', 'M', 'C', 'P', 'u', 'S', 'E', '‘', '*', '°', 'f', '#', 'F', '£', 'K', 'O', '₂', 'w', 'd', 'x', 'B', 'T', 'R', 'H', 'L', 'W', '×', 'फ़']

Char to int:  {'<UNK>': tensor(0), '<pad>': tensor(1), '<eos>': tensor(2), 'क': tensor(3), 'ा': tensor(4), 'ं': tensor(5), 'ग': tensor(6), '्': tensor(7), 'र': tensor(8), 'े': tensor(9), 'स': tensor(10),

In [7]:
def encode_text(input_text,vocab):
    # Encode the input_text replacing the char by its integer number based on the dictionary vocab
    output = torch.tensor([vocab.char2int.get(character, 0) for character in input_text])
    return output

In [8]:
encoded_dataset = encode_text(dataset, vocab)
print("Original dataset : \n",encoded_dataset[:100])
type(encoded_dataset)

Original dataset : 
 tensor([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12,  9, 13,  4, 11, 14, 15, 16, 17,
         5, 18,  8, 11, 10, 17,  5, 19, 11,  3, 20, 11, 21,  5, 16,  4, 14, 11,
        22,  9,  5, 11, 23,  8, 11,  3,  9, 11, 24,  5, 18,  8, 11,  6, 25, 15,
        20, 11, 22,  4,  8,  3,  8, 11,  3, 20, 11,  6, 26, 11, 19, 13,  7, 27,
         4, 11, 28, 11,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,  9, 13,  4, 11,
        14, 15, 16, 17,  5, 18,  8, 11, 10, 17])


torch.Tensor

In [9]:
int(len(encoded_dataset)/32)

54793

In [36]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size,num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.embed = nn.Embedding(input_size,hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers,
                           batch_first = True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, hidden, cell):
        out = self.embed(x)
#         print(out.shape,'-'*4)
        out, (hidden, cell) = self.lstm(out.unsqueeze(1), (hidden,cell))
        out = self.fc(out.reshape(out.shape[0], -1))
        return out,(hidden, cell)
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        return hidden, cell
    


In [37]:
encoded_dataset= encoded_dataset[:10000]

In [38]:
def chunk_len_calculator(batch_size):
    chunk_len = int(len(encoded_dataset) / batch_size)
    if chunk_len <= 3 :
        print("Please provide the smaller batch size")
        return None
    else:
        return chunk_len
    

In [39]:
input_size = len(vocab.unique_chars) #embedding size
output_size = len(vocab.unique_chars) # vocab size
num_epochs = 100
batch_size = 64
print_every = 5
hidden_size = 256
chunk_len = chunk_len_calculator(batch_size)
num_layers = 2
lr = 0.002

In [40]:
def get_random_batch(chunk_len,batch_size):

    text_input = torch.zeros(batch_size, chunk_len)
    text_target = torch.zeros(batch_size, chunk_len)

    for i in range(batch_size):
        start_idx = random.randint(0,len(encoded_dataset)-chunk_len-1)
        end_idx = start_idx + chunk_len + 1
        text_str = encoded_dataset[start_idx:end_idx]
        text_input[i,:] = text_str[:-1]
        text_target[i,:] = text_str[1:]

    return text_input.long(), text_target.long()

In [41]:
a,b = get_random_batch(chunk_len=chunk_len,batch_size=batch_size)

In [42]:
encoded_dataset[9844:10003]

tensor([ 9, 11, 22, 38, 39,  7, 27, 22,  5, 13,  7,  8, 20, 11, 12, 20, 13, 20,
        44, 11,  3, 38, 22,  4,  8, 11,  3,  4, 11, 35,  3, 11, 29, 20, 51, 17,
        27, 25, 11, 10,  4, 22, 12,  9, 11, 34, 27,  4, 11, 19, 40, 11, 16, 17,
        10, 22,  9,  5, 11, 29, 19, 11, 22,  5, 13,  7,  8, 20, 11, 24, 44, 25,
         3, 11, 77, 68, 53,  8, 20, 11,  3, 20, 11,  6,  8,  7, 18, 12, 11, 21,
         3, 51, 58,  3,  8, 11, 35,  3, 11, 21, 13,  7,  8,  3,  4,  8, 11,  3,
         9, 11, 22,  4, 46,  9, 11, 10,  9, 11, 30, 12,  3,  4, 11, 22,  4, 46,
         4, 11, 36,  3,  8,  4, 13,  9, 11, 19, 38, 35, 11, 18, 17, 39, 11,  8,
        19,  9, 11, 19, 40,  5, 31, 11, 18,  8, 24, 10])

In [43]:
a.shape

torch.Size([64, 156])

In [44]:
model = LSTM(input_size=input_size,hidden_size=hidden_size,
            num_layers=num_layers,output_size=output_size)

model = model.to(device)
print(model)
# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

LSTM(
  (embed): Embedding(148, 256)
  (lstm): LSTM(256, 256, num_layers=2, batch_first=True)
  (fc): Linear(in_features=256, out_features=148, bias=True)
)


In [45]:
def train_model(model,optimizer,loss_fn,
               batch_size,epochs,chunk_len,device,print_every):

    print("-> Starting Training")
    count = 1
    for epoch in tqdm.tqdm(range(epochs)):
#                       desc = "Epoch {}/{}".format(count,epochs)):
        inp,target = get_random_batch(chunk_len=chunk_len,batch_size=batch_size)
#         print(inp.shape)
        hidden, cell = model.init_hidden(batch_size=batch_size)
#         print(hidden.shape)
        model.zero_grad()
        loss = 0
        inp = inp.to(device)
        target = target.to(device)

        for c in range(chunk_len):
            output, (hidden, cell) = model(inp[:,c],
                                             hidden,cell)
            loss += criterion(output,target[:, c])

        loss.backward()
        optimizer.step()
        loss = loss.item()/chunk_len

        if epoch % print_every ==0:
#             print(f'Loss:{loss}')
            print('Epoch: {}/{}.......'.format(count,epochs),end=' ')
            print("Train Loss: {:.4f}".format(np.mean(loss)),end=' ')
        count += 1
#                 print(self.generate())


In [46]:
train_model(model=model,batch_size=batch_size,
            chunk_len=chunk_len,device=device,
            epochs=500,loss_fn=criterion,
            optimizer=optimizer,print_every=1)

-> Starting Training


  0%|▏                                                                                 | 1/500 [00:02<22:40,  2.73s/it]

Epoch: 1/500....... Train Loss: 4.9863 

  0%|▎                                                                                 | 2/500 [00:05<22:45,  2.74s/it]

Epoch: 2/500....... Train Loss: 4.8963 

...

Epoch: 498/500....... Train Loss: 0.1098 

100%|███████████████████████████████████████████████████████████████████████████████▊| 499/500 [44:09<00:02,  2.71s/it]

Epoch: 499/500....... Train Loss: 0.1176 

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [44:12<00:00,  5.31s/it]

Epoch: 500/500....... Train Loss: 0.1160 

In [47]:
def generate_text(model,
                 initial_str = 'शिकायत',
                 predict_len = 190,
                 temperature = 0.8,
                 vocab = vocab):
    model.eval()
    hidden, cell = model.init_hidden(batch_size =1)
    initial_input = encode_text(initial_str, vocab=vocab)
    predicted = initial_str
    
    #warm up stage
    for p in range(len(initial_str) - 1):
        _,(hidden,cell) = model(initial_input[p].view(1).to(device),
                                hidden, cell)
    last_char = initial_input[-1]
    
    for p in range(predict_len):
        output, (hidden,cell) = model(last_char.view(1).to(device),
                                     hidden,cell)
        output_dist = output.data.view(-1).div(temperature).exp()
        top_char = torch.multinomial(output_dist,1)[0]
        predicted_char = vocab.unique_chars[top_char]
        predicted += predicted_char
        last_char = vocab.char2int[predicted_char]
        
    return predicted
    

In [55]:
gen = generate_text(model,
                 initial_str = 'कई तरह की गतिविधियों' ,
                 predict_len = 190,
                 temperature = 1,
                 vocab = vocab)
print(gen)

कई तरह की गतिविधियों का 33% आर्हत्धान लो' किया गयायारु कर दिया और राज्यसभा के सत्रों का आयोजन नहीं होगा लेकिन ऐतिहासिक और पुरातात्विक महत्व के कारण )िल की तहाया गया था: मणिपुर में मारे गए सैनिक की पत्नी : मणिपु


## Plotting Graph

In [ ]:
from torchviz import make_dot
# x = torch.randint(0, input_size, (1, 5))  # Replace with your input shape
hidden, cell = model.init_hidden(batch_size)
y, _ = model(a[:,1],hidden, cell)
dot = make_dot(y, params=dict(model.named_parameters()))
dot.render("lstm_model_graph", format="png")  # Save the graph as an image
dot